# About

Code for https://towardsdatascience.com/how-to-handle-large-datasets-in-python-with-pandas-and-dask-34f43a897d55

This is only works for Python 3.7 (3.8 or later won't work given that `tables` only supports 3.7)

## Data

To demonstrate the power of Pandas/Dask, I chose chose an open-source dataset from Wikipedia about the source of the site’s visitors. You can get the ‘clickstream’ tables (in .tsv) here: https://dumps.wikimedia.org/other/clickstream/.
The clickstream data contains 4 main columns:
- ‘Prev’ — the site from which the visitor came (I renamed it to ‘coming_from’)
- ‘curr’ — the target article page (renamed to ‘article’)
- ‘type’ — this column describes the type of referral, for example, an external link (I renamed it to ‘referral_type’)
- ’n’ — the number of visits


Next, I came up with a few questions to play around with my dataset and check whether the combination of Pandas and Dask does its job:
- Which links do people click on most often in a given article?
- What are the most popular articles users access from all the external search engines?
- What percentage of visitors to a given article page have clicked on a link to get there?
- What is the most common source of visits for each article? (displayed in a single table)
The dataset size is 1.4 Gb, so it carries significant risk of memory overload. That’s why I split the study into two parts.
First, I implemented the analysis on a limited data subset using just the Pandas library. Then I attempted to do exactly the same on the full set using Dask.

Run the following in the terminal to get the large dataset (about 388m zipped and 1.47G unzipped)

you need the following first:

```
brew install wget
brew install gunzip
```

```
mkdir ./data
cd data
wget https://dumps.wikimedia.org/other/clickstream/2018-12/clickstream-enwiki-2018-12.tsv.gz
gunzip clickstream-enwiki-2018-12.tsv.gz
```


In [25]:
import pandas as pd

df = pd.read_csv('data/clickstream-enwiki-2018-12.tsv', delimiter='\t',
    names=['coming_from', 'article', 'referrer_type', 'n'],
    dtype={
        'referrer_type': 'category',
        'n': 'uint32'}
)

In [15]:
# sample
df = df.iloc[:100_000]

In [26]:
top_links = df.loc[
    df['referrer_type'].isin(['link']),   
    ['coming_from','article', 'n']
]\
.groupby(['coming_from', 'article'])\
.sum()\
.sort_values(by='n', ascending=False)

top_links[:20]

n
coming_from       article                                       
Jason_Momoa       Lisa_Bonet                             1166522
Priyanka_Chopra   Nick_Jonas                              596798
Bird_Box          Bird_Box_(film)                         508241
Nick_Jonas        Priyanka_Chopra                         493898
George_H._W._Bush George_W._Bush                          453809
                  Pauline_Robinson_Bush                   419132
                  Barbara_Bush                            412565
George_W._Bush    George_H._W._Bush                       393452
George_H._W._Bush Dorothy_Bush_Koch                       377785
Emily_Atack       Kate_Robbins                            354577
Macaulay_Culkin   Brenda_Song                             346110
George_H._W._Bush Marvin_Bush                             339241
                  Neil_Bush                               330132
Lisa_Bonet        Jason_Momoa                             321970
Bird_Box_(film)   Trevante_Rhodes                         309028
Penny_Marshall    Tracy_Reiner                            281911
George_H._W._Bush Prescott_Bush                           253973
Welcome_to_Marwen Marwencol_(film)                        253955
Lisa_Bonet        Zoë_Kravitz                             250392
2.0_(film)        List_of_highest-grossing_Indian_films   249322

In [18]:
from dask import dataframe as dd

dfd = dd.read_csv(
    'data/clickstream-enwiki-2018-12.tsv', 
    delimiter='\t',
    names=['coming_from', 'article', 'referrer_type', 'n'],
    dtype={
        'referrer_type': 'category',
        'n': 'uint32'},
    blocksize=64000000 # = 64 Mb chunks
)

In [19]:
top_links_grouped_dask = dfd.loc[
    dfd['referrer_type'].isin(['link']), 
    ['coming_from','article', 'n']]\
        .groupby(['coming_from', 'article'])

In [20]:
store = pd.HDFStore('./data/clickstream_store.h5')

ImportError: Missing optional dependency 'tables'.  Use pip or conda to install tables.

In [21]:
top_links_dask = top_links_grouped_dask.sum().nlargest(20, 'n')

In [22]:
top_links_dask.compute()

n
coming_from       article                                       
Jason_Momoa       Lisa_Bonet                             1166522
Priyanka_Chopra   Nick_Jonas                              596798
Bird_Box          Bird_Box_(film)                         508241
Nick_Jonas        Priyanka_Chopra                         493898
George_H._W._Bush George_W._Bush                          453809
                  Pauline_Robinson_Bush                   419132
                  Barbara_Bush                            412565
George_W._Bush    George_H._W._Bush                       393452
George_H._W._Bush Dorothy_Bush_Koch                       377785
Emily_Atack       Kate_Robbins                            354577
Macaulay_Culkin   Brenda_Song                             346110
George_H._W._Bush Marvin_Bush                             339241
                  Neil_Bush                               330132
Lisa_Bonet        Jason_Momoa                             321970
Bird_Box_(film)   Trevante_Rhodes                         309028
Penny_Marshall    Tracy_Reiner                            281911
George_H._W._Bush Prescott_Bush                           253973
Welcome_to_Marwen Marwencol_(film)                        253955
Lisa_Bonet        Zoë_Kravitz                             250392
2.0_(film)        List_of_highest-grossing_Indian_films   249322